# ОТЧЕТ

#### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? ####

Качество = 0.72
Это сравнимо с градиентным бустингом, где количество деревьев превышает 150
Логистическая регрессия работает быстрее градиентного бустинга    

#### 2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Качество после удаления категориальных признаков не изменилось   
Следовательно, категориальные признаки не влияли на кросс-валидацию   

#### 3. Сколько различных идентификаторов героев существует в данной игре?

Всего героев в игре: 112  

#### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

0.75. Качество увеличилось.    
Как оказалось, категориальных признаки имели большой вес, и стоило не удалять их а зменить на числовые аналоги

#### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Минимум = 0.009    
Максимум = 0.9965

# Подход 2: логистическая регрессия #

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint 
import time
import datetime
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [3]:
featuresTrain = pd.read_csv('data/features.csv', index_col='match_id')

In [4]:
featuresTrain = featuresTrain.drop([
		'duration', 
		'tower_status_radiant', 
		'tower_status_dire', 
		'barracks_status_radiant', 
		'barracks_status_dire'
		], 
		axis=1)

In [5]:
def cleanEmptyColumn(df):
	# Замена всех пропусков на 0
	df = df.fillna(0)
	return df

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? ###



In [6]:
def logisticRegression(XTrain, yTrain):
	scaler = StandardScaler()
	XTrain = scaler.fit_transform(XTrain)
	kf = KFold(yTrain.size, n_folds=5, shuffle=True, random_state=241)
	cRange = range(-5, 5)
	for c in cRange:
		c = 10**c
		print('Параметр С: ',c)
		model = LogisticRegression(penalty='l2', C=c, random_state=241)
		startTime = datetime.datetime.now()
		scores = cross_val_score(model, XTrain, yTrain.values.ravel(), cv=kf, scoring='roc_auc')
		print('Время обучения:', datetime.datetime.now() - startTime)
		print(scores)
		print("_____")

In [7]:
trainData = cleanEmptyColumn(featuresTrain)
XTrain = trainData.drop('radiant_win',1)
yTrain = trainData['radiant_win'].to_frame()
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:04.124701
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
_____
Параметр С:  0.0001
Время обучения: 0:00:05.589951
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
_____
Параметр С:  0.001
Время обучения: 0:00:09.860917
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
_____
Параметр С:  0.01
Время обучения: 0:00:13.049704
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
_____
Параметр С:  0.1
Время обучения: 0:00:13.571960
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
_____
Параметр С:  1
Время обучения: 0:00:13.283247
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
_____
Параметр С:  10
Время обучения: 0:00:14.023975
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
_____
Параметр С:  100
Время обучения: 0:00:12.991622
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
_____
Параметр С:  1000
Время обучения: 0:00:13.795557
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
_

#### ОТВЕТ:     ####
С = 0.01 - лучший парамерр регуляризации   
Качество = 0.72

Это сравнимо с градиентным бустингом, где количество деревьев превышает 150   
Логистическая регрессия работает быстрее градиентного бустинга

### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить? ###

In [8]:
def cleanCategory(X):
    X.drop('lobby_type',1)
    X.drop('r1_hero',1)
    X.drop('r2_hero',1)
    X.drop('r3_hero',1)
    X.drop('r4_hero',1)
    X.drop('r5_hero',1)
    X.drop('d1_hero',1)
    X.drop('d2_hero',1)
    X.drop('d3_hero',1)
    X.drop('d4_hero',1)
    X.drop('d5_hero',1)
    return X

In [9]:
XTrain = cleanCategory(XTrain)

In [10]:
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:04.068835
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
_____
Параметр С:  0.0001
Время обучения: 0:00:05.405390
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
_____
Параметр С:  0.001
Время обучения: 0:00:09.573467
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
_____
Параметр С:  0.01
Время обучения: 0:00:13.255760
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
_____
Параметр С:  0.1
Время обучения: 0:00:14.784955
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
_____
Параметр С:  1
Время обучения: 0:00:19.835372
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
_____
Параметр С:  10
Время обучения: 0:00:15.465531
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
_____
Параметр С:  100
Время обучения: 0:00:12.278096
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
_____
Параметр С:  1000
Время обучения: 0:00:11.058031
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
_

#### ОТВЕТ ####
Качество после удаления категориальных признаков не изменилось      
Следовательно, категориальные признаки не влияли на кросс-валидацию 

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts). ###

In [11]:
def numHeroes(heroes):
	print('Всего героев в игре:', len(heroes))

In [12]:
heroes = pd.read_csv('data/dictionaries/heroes.csv')
numHeroes(heroes)

Всего героев в игре: 112


### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа. ###

In [13]:
N = heroes.shape[0]
X_pick = np.zeros((XTrain.shape[0], N))

for i, match_id in enumerate(XTrain.index):
    for p in range(5):
        X_pick[i, XTrain.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, XTrain.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1



### 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить? ###

In [14]:
X_pick_transp = np.transpose(X_pick)
for i in range(0, N):
    XTrain["bow_"+str(i)] = pd.Series(X_pick_transp[i], index=XTrain.index)
    
logisticRegression(XTrain,yTrain)

Параметр С:  1e-05
Время обучения: 0:00:07.994615
[0.71902994 0.71120697 0.71507984 0.71291814 0.71598965]
_____
Параметр С:  0.0001
Время обучения: 0:00:10.365467
[0.74673258 0.74001936 0.74260214 0.74026026 0.74430669]
_____
Параметр С:  0.001
Время обучения: 0:00:17.039373
[0.75503591 0.74925153 0.75165321 0.74902422 0.75306145]
_____
Параметр С:  0.01
Время обучения: 0:00:23.080101
[0.75509143 0.74979078 0.75201826 0.74951065 0.75309794]
_____
Параметр С:  0.1
Время обучения: 0:00:28.741750
[0.75500689 0.74982166 0.75197248 0.74953636 0.75294704]
_____
Параметр С:  1
Время обучения: 0:00:27.879960
[0.75499645 0.74982521 0.75196506 0.74953762 0.75292372]
_____
Параметр С:  10
Время обучения: 0:00:24.998066
[0.75499654 0.74982551 0.75196402 0.74953846 0.75292095]
_____
Параметр С:  100
Время обучения: 0:00:24.644725
[0.75499658 0.74982553 0.75196381 0.74953847 0.7529206 ]
_____
Параметр С:  1000
Время обучения: 0:00:30.778881
[0.7549965  0.74982559 0.75196385 0.74953845 0.75292055]
_

#### ОТВЕТ ####

0.76. Качество увеличилось. Как оказалось, категориальных признаки имели большой вес, и стоило не удалять их а зменить на числовые аналоги

### 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной). ###

In [15]:
featuresTest = pd.read_csv('data/features_test.csv', index_col='match_id')
X_pick = np.zeros((featuresTest.shape[0], N))
for i, match_id in enumerate(featuresTest.index):
    for p in range(5):
        X_pick[i, featuresTest.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, featuresTest.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X = cleanCategory(featuresTest)

X_pick_transp = np.transpose(X_pick)
for i in range(0, N):
    X["bow_"+str(i)] = pd.Series(X_pick_transp[i], index=featuresTest.index)

X = cleanEmptyColumn(X)
scaler = StandardScaler()
XTest = scaler.fit_transform(X)
XTrain = scaler.fit_transform(XTrain)

model = LogisticRegression(penalty='l2', C=0.1, random_state=241)
model.fit(XTrain,yTrain.values.ravel())

probPos = model.predict_proba(XTest)
    
print('min ' + str(min(probPos[:, 1])))
print('max ' + str(max(probPos[:, 1])))

min 0.008901752967975662
max 0.9965022067810528
